In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

In [ ]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
class EnhancedMedicalChatbot:
    def __init__(self, data, model_name='emilyalsentzer/Bio_ClinicalBERT'):
        self.data = data
        # Initialize BERT components
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.disease_embeddings = None
        self.context = None
        self.user_name = None


        # Disease information dictionary
        self.disease_info = {
            'Dengue': {
                'description': "Dengue is caused by the bite of an infected Aedes mosquito. It leads to high fever, severe joint and muscle pain, and skin rashes.",
                'treatment': [
                    "Drink plenty of fluids to stay hydrated.",
                    "Rest as much as possible.",
                    "Take paracetamol for fever and pain.",
                    "Avoid aspirin or ibuprofen.",
                    "Seek immediate medical attention for severe symptoms."
                ],
                'precautions': [
                    "Use mosquito repellents and nets.",
                    "Wear long-sleeved clothing to prevent bites.",
                    "Eliminate standing water around your home."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Psoriasis': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Keep your skin moisturized",
                  "Avoid triggers like stress and certain foods",
                  "Use prescribed medications as directed",
                  "Consider phototherapy under medical supervision",
                  "See a dermatologist for proper treatment"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            ##
            'Varicose Veins': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Exercise regularly to improve circulation",
                  "Elevate your legs when resting",
                  "Avoid standing or sitting for long periods",
                  "Wear compression stockings as recommended",
                  "Consult a healthcare provider for severe cases"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Typhoid': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Drink plenty of fluids to prevent dehydration",
                  "Follow a bland diet during recovery",
                  "Take antibiotics as prescribed by your doctor",
                  "Maintain proper hygiene to avoid reinfection",
                  "Seek medical attention if symptoms worsen"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Chicken pox': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Avoid scratching the rash to prevent scarring",
                  "Use calamine lotion to soothe itching",
                  "Stay hydrated and get plenty of rest",
                  "Isolate yourself to avoid spreading the infection",
                  "Consult a doctor if complications arise"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
              'Impetigo': {
              'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
              'treatment': [
                  "Keep the affected area clean and covered",
                  "Avoid touching or scratching the sores",
                  "Use prescribed antibiotic ointments",
                  "Wash hands frequently to prevent spreading",
                  "Consult a doctor for persistent infections"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Fungal infection': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Keep the affected area clean and dry",
                  "Use antifungal creams as prescribed",
                  "Avoid sharing personal items",
                  "Wear breathable clothing",
                  "Consult a dermatologist if the infection persists"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Common Cold': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Drink warm fluids to soothe your throat",
                  "Get plenty of rest",
                  "Use over-the-counter medications for relief",
                  "Maintain good hygiene to prevent spreading",
                  "Consult a doctor if symptoms worsen"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Pneumonia': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Dimorphic Hemorrhoids': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Eat a high-fiber diet to prevent constipation",
                  "Drink plenty of water",
                  "Use over-the-counter creams for relief",
                  "Avoid straining during bowel movements",
                  "Consult a doctor for severe or persistent cases"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Arthritis': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Engage in low-impact exercises like swimming",
                  "Use heat or cold therapy for pain relief",
                  "Maintain a healthy weight",
                  "Take medications as prescribed",
                  "Consult a rheumatologist for specialized care"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Acne': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Wash your face twice daily with a gentle cleanser",
                  "Avoid picking or squeezing pimples",
                  "Use non-comedogenic skincare products",
                  "Apply topical treatments as prescribed",
                  "Consult a dermatologist for severe acne"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Bronchial Asthma': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Use inhalers as prescribed",
                  "Avoid triggers like allergens and smoke",
                  "Practice breathing exercises",
                  "Stay physically active with caution",
                  "Consult a pulmonologist for management"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Hypertension': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Migraine': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Cervical spondylosis': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Jaundice': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Malaria': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'urinary tract infection': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'allergy': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'gastroesophageal reflux disease': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'drug reaction': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'peptic ulcer disease': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'diabetes': {
                'description': "Psoriasis is a skin disease that causes a rash with itchy, scaly patches, most commonly on the knees, elbows, trunk and scalp.",
                'treatment': [
                  "Take prescribed antibiotics or antiviral medications",
                  "Rest and drink plenty of fluids",
                  "Avoid smoking and secondhand smoke",
                  "Use a humidifier to ease breathing",
                  "Seek medical attention for severe symptoms"
                ],
                'precautions': [
                    "Prevent skin injuries.",
                    "Improve Diet.",
                    "Avoid UV light."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            }
        }

        # Encouraging messages and jokes
        self.jokes = [
            "Why don't skeletons fight each other? They don't have the guts! But you have it to fight with your disease.",
            "You're stronger than you think! Keep fighting, and this too shall pass.",
            "Stay happy and healthy. Let me tell you a joke, Laughter is the best medicine—unless you're dealing with an actual doctor!"
        ]

        self.prepare_data()

    def prepare_data(self):
        """Prepare disease embeddings using ClinicalBERT"""
        disease_descriptions = []
        disease_labels = []

        # Combine symptoms for each disease
        for disease in self.data['label'].unique():
            symptoms = self.data[self.data['label'] == disease]['text'].tolist()
            combined_symptoms = ' '.join(symptoms)
            disease_descriptions.append(combined_symptoms)
            disease_labels.append(disease)

        # Generate embeddings
        self.disease_embeddings = self.get_bert_embeddings(disease_descriptions)
        self.disease_labels = disease_labels

    def get_bert_embeddings(self, texts):
        """Generate BERT embeddings for input texts"""
        embeddings = []

        with torch.no_grad():
            for text in texts:
                inputs = self.tokenizer(text,
                                      return_tensors="pt",
                                      max_length=512,
                                      truncation=True,
                                      padding=True)

                outputs = self.model(**inputs)
                embedding = outputs.last_hidden_state[:, 0, :].numpy()
                embeddings.append(embedding[0])

        return np.array(embeddings)

    def predict_disease(self, user_input):
        """Predict disease based on user symptoms"""
        # Get embedding for user input
        user_embedding = self.get_bert_embeddings([user_input])

        # Calculate similarities
        similarities = cosine_similarity(user_embedding, self.disease_embeddings)

        # Get top matches
        top_indices = similarities[0].argsort()[-3:][::-1]
        predictions = [
            (self.disease_labels[idx], similarities[0][idx])
            for idx in top_indices
        ]

        return predictions

    def get_response(self, user_input):
        """Generate chatbot response based on user input"""
        user_input = user_input.lower().strip()

        # Handle exit commands
        if "quit" in user_input or "bye" in user_input:
            return f"Goodbye {self.user_name if self.user_name else 'user'}! Take care and stay healthy."

        # Handle name introduction
        name_patterns = [
        "i am ([a-zA-Z]+)$",  # Matches "i am John"
        "my name is ([a-zA-Z]+)$",  # Matches "my name is John"
        "^([a-zA-Z]+) here$"  # Matches "John here"
    ]
        for pattern in name_patterns:
          match = re.search(pattern, user_input)
          if match:
            self.user_name = match.group(1)
            return f"Hello {self.user_name}! How are you feeling today?"

        # Handle context-specific queries
        if self.context:
            if "what is it" in user_input or "tell me about it" in user_input:
                return self.disease_info[self.context]['description']
            if "treatment" in user_input:
                return "\n".join(self.disease_info[self.context]['treatment'])
            if "precaution" in user_input:
                return "\n".join(self.disease_info[self.context]['precautions'])
            if "severity" in user_input or "severe" in user_input:
                return self.disease_info[self.context]['severity']
            if "will i die" in user_input or "i am feeling low" in user_input or "i am feeling scared" in user_input:
                return f"Don't worry, {self.user_name}! {np.random.choice(self.jokes)}"

        # Get disease predictions
        predictions = self.predict_disease(user_input)
        predicted_disease, confidence = predictions[0]

        # Set context and generate response
        if confidence < 0.3:
            return "I'm sorry, I couldn't understand your symptoms clearly. Could you please provide more details about what you're experiencing?"

        self.context = predicted_disease
        response = f"Based on your symptoms, {self.user_name if self.user_name else 'user'}, it looks like you might have {predicted_disease}.\n"

        if predicted_disease in self.disease_info:
            response += "Here are some important tips:\n"
            response += "\n".join(self.disease_info[predicted_disease]['treatment'])

        response += "\n\nPlease note: This is not a definitive diagnosis. Always consult a qualified healthcare professional for proper medical evaluation."
        return response


In [ ]:
def initialize_chatbot(data_path):
    """Initialize and return the chatbot instance"""
    print("Loading medical knowledge base and initializing the chatbot...")
    data = pd.read_csv(data_path)
    return EnhancedMedicalChatbot(data)

In [ ]:
# Main execution
if __name__ == "__main__":
    # Initialize the chatbot
    input_dataset = '/content/drive/MyDrive/715 project/Disease Text Data/Data.csv'  # Update with your actual path
    chatbot = initialize_chatbot(input_dataset)

    # Start interactive loop
    print("Hello! I am your medical assistant. How can I help you today?")
    print("You can introduce yourself by saying 'My name is...' or 'I am...'")
    print("To exit, simply type 'quit' or 'bye'")

    while True:
        try:
            user_input = input("\nYou: ")
            response = chatbot.get_response(user_input)
            print(f"\nBot: {response}")

            if "Goodbye" in response:
                break

        except Exception as e:
            print(f"\nBot: I apologize, but I encountered an error. Please try rephrasing your question or describing your symptoms differently.")
            print(f"Error details: {str(e)}")

Loading medical knowledge base and initializing the chatbot...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Hello! I am your medical assistant. How can I help you today?
You can introduce yourself by saying 'My name is...' or 'I am...'
To exit, simply type 'quit' or 'bye'

You: bye

Bot: Goodbye user! Take care and stay healthy.


In [ ]:
!pip install gradio

import gradio as gr
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
# Keep your existing EnhancedMedicalChatbot class exactly as is
# [Previous EnhancedMedicalChatbot class code remains the same]

class EnhancedMedicalChatbot:
    def __init__(self, data, model_name='emilyalsentzer/Bio_ClinicalBERT'):
        self.data = data
        # Initialize BERT components
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.disease_embeddings = None
        self.context = None
        self.user_name = None


        # Disease information dictionary
        self.disease_info = {
            'Dengue': {
                'description': "Dengue is caused by the bite of an infected Aedes mosquito. It leads to high fever, severe joint and muscle pain with skin rashes.",
                'treatment': [
                  "Drink plenty of fluids to stay hydrated.",
                  "Rest as much as possible.",
                  "Take paracetamol for fever and pain.",
                  "Avoid aspirin or ibuprofen.",
                  "Seek immediate medical attention for severe symptoms."
                ],
                'precautions': [
                  "Use mosquito repellents and nets.",
                  "Wear long-sleeved clothing to prevent bites.",
                  "Eliminate standing water around your home."
                ],
                'severity': "Moderate to severe. Can lead to life-threatening complications if untreated."
            },
            'Psoriasis': {
                'description': "Psoriasis is a chronic autoimmune skin condition that accelerates the life cycle of skin cells, leading to the buildup of cells on the surface of the skin. This causes red, inflamed, itchy, and scaly patches, often appearing on the knees, elbows, scalp, lower back, and nails. Psoriasis can vary in severity from small localized patches to extensive coverage, and it may flare periodically. In some cases, it is associated with joint pain and stiffness, a condition known as psoriatic arthritis.",
                'treatment': [
                  "Topical treatments: Corticosteroids, vitamin D analogs, retinoids, and salicylic acid-based creams to reduce inflammation and slow skin cell turnover.",
                  "Phototherapy: Controlled exposure to natural or artificial ultraviolet (UV) light to reduce symptoms and slow disease progression.",
                  "Systemic treatments: For severe cases, oral or injectable medications such as methotrexate, cyclosporine, or biologics (like TNF inhibitors) may be prescribed to target the immune system.",
                  "Moisturize regularly to prevent dryness and reduce scaling.",
                  "Avoid triggers such as stress, smoking, alcohol, and cold weather.",
                  "Seek medical attention for persistent or worsening symptoms, especially if joint pain or swelling occurs."
                ],
                'precautions': [
                  "Protect skin from injury, as trauma can trigger flare-ups (known as the Koebner phenomenon).",
                  "Maintain a healthy, anti-inflammatory diet rich in omega-3 fatty acids, fruits, and vegetables.",
                  "Manage stress through meditation, yoga, or other relaxation techniques.",
                  "Limit alcohol intake and avoid smoking, as these can exacerbate symptoms.",
                  "Stay hydrated and use gentle, fragrance-free skin products.",
                  "Wear loose, soft clothing to avoid irritation and prevent further skin damage.",
                  "Avoid excessive sun exposure, but ensure adequate vitamin D levels with safe sun practices."
                ],
                'severity': "Mild to severe. Psoriasis can significantly impact quality of life and, if left untreated, may lead to psoriatic arthritis, which can cause permanent joint damage and disability."
            },
            ##
            'Varicose Veins': {
                'description': "Varicose veins are swollen, twisted veins that are visible just under the surface of the skin, most commonly affecting the legs. They occur when faulty valves in the veins allow blood to pool or flow backward, causing the veins to enlarge. Symptoms may include aching pain, heaviness, burning, throbbing, muscle cramping, and swelling in the lower legs. In more severe cases, varicose veins can lead to skin discoloration, ulcers, or bleeding.",
                'treatment': [
                  "Compression stockings: Wear to improve circulation and reduce swelling.",
                  "Exercise regularly: Walking or low-impact activities help improve blood flow.",
                  "Leg elevation: Elevate legs to reduce pressure and swelling.",
                  "Sclerotherapy: Injection of a solution that scars and closes smaller veins.",
                  "Laser treatments: Non-invasive method to close off smaller varicose veins.",
                  "Surgical procedures: In severe cases, options include vein stripping or ligation to remove or close large varicose veins."
                ],
                'precautions': [
                  "Avoid prolonged standing or sitting, as it can exacerbate symptoms.",
                  "Maintain a healthy weight to reduce pressure on veins.",
                  "Regular exercise to promote good circulation.",
                  "Avoid crossing legs while sitting to prevent restricted blood flow.",
                  "Wear loose clothing that doesn’t constrict veins.",
                  "Stay hydrated and consume foods rich in fiber to prevent constipation, which can contribute to vein pressure."
                ],
                'severity': "Mild to moderate. While varicose veins are generally not dangerous, complications such as ulcers, bleeding, or deep vein thrombosis (DVT) can occur if left untreated."
            },

            'Typhoid': {
                'description': "Typhoid fever is a bacterial infection caused by Salmonella Typhi. It spreads through contaminated food and water and is more common in areas with poor sanitation. Symptoms typically develop 1-3 weeks after exposure and may include prolonged high fever, weakness, stomach pain, headache, loss of appetite, constipation or diarrhea, and rose-colored spots on the chest. If left untreated, typhoid can lead to serious complications such as intestinal perforation or hemorrhage.",
                'treatment': [
                  "Antibiotics: Prescribed to kill the Salmonella bacteria (commonly ciprofloxacin or azithromycin).",
                  "Hydration therapy: Drink plenty of fluids to prevent dehydration.",
                  "Rest: Adequate rest to support recovery.",
                  "Healthy diet: Light, easily digestible meals to maintain energy.",
                  "Hospitalization: In severe cases, hospitalization may be required for intravenous antibiotics and fluid replacement.",
                  "Surgery: In rare cases of intestinal perforation, surgical intervention may be necessary."
                ],
                'precautions': [
                  "Practice good hygiene: Wash hands thoroughly with soap and water, especially before eating.",
                  "Drink safe water: Boil or treat water before drinking, and avoid ice in drinks.",
                  "Eat safe food: Consume thoroughly cooked food and avoid raw fruits or vegetables that cannot be peeled.",
                  "Vaccination: Get vaccinated when traveling to high-risk areas.",
                  "Avoid close contact: Steer clear of individuals who are infected.",
                  "Maintain proper sanitation: Ensure waste is disposed of safely and clean toilets are used."
                ],
                'severity': "Moderate to severe. Typhoid fever can lead to life-threatening complications if not treated promptly. Early diagnosis and antibiotic treatment significantly reduce the risk of complications."

            },
            'Chicken pox': {
                'description': "Chickenpox is a highly contagious viral infection caused by the varicella-zoster virus. It primarily affects children but can occur at any age. It is characterized by an itchy rash that develops into fluid-filled blisters, which eventually scab over. Other symptoms include fever, fatigue, loss of appetite, and headache. The rash usually appears first on the chest, back, and face, then spreads to the rest of the body. Severe cases can lead to complications such as bacterial skin infections, pneumonia, or encephalitis.",
                'treatment': [
                  "Antiviral medications: Prescribed in severe cases or for high-risk individuals to reduce the severity and duration of the illness.",
                  "Antihistamines: Help reduce itching and discomfort.",
                  "Calamine lotion and oatmeal baths: Soothe the skin and relieve itching.",
                  "Acetaminophen or ibuprofen: Used to reduce fever and pain (avoid aspirin in children due to the risk of Reye's syndrome).",
                  "Hydration and rest: Encourage fluid intake and sufficient rest to aid recovery.",
                  "Prevent scratching: Trim nails to prevent skin infections from scratching blisters."
                ],
                'precautions': [
                  "Vaccination: The varicella vaccine is highly effective in preventing chickenpox or reducing its severity.",
                  "Avoid close contact: Stay away from infected individuals to prevent transmission.",
                  "Maintain good hygiene: Regular handwashing and disinfecting surfaces can limit the spread of the virus.",
                  "Isolate infected individuals: Keep those with chickenpox at home until all blisters have scabbed over.",
                  "Boost immunity: Maintain a healthy lifestyle to strengthen the immune system.",
                  "Monitor for complications: Seek medical attention if symptoms worsen or if breathing difficulties develop."
                ],
                'severity': "Mild to moderate. While chickenpox is typically mild, complications can arise, particularly in adults, pregnant women, newborns, and individuals with weakened immune systems. Vaccination greatly reduces the risk of severe disease."


            },
              'Impetigo': {
                'description': "Impetigo is a highly contagious bacterial skin infection that primarily affects infants and young children. It is caused by Staphylococcus aureus or Streptococcus pyogenes. Impetigo typically appears as red sores or blisters that rupture, ooze, and form honey-colored crusts. The infection commonly occurs around the nose, mouth, hands, and feet but can spread to other areas through touch or shared items. While impetigo is usually mild, it can lead to complications such as cellulitis or kidney inflammation (post-streptococcal glomerulonephritis) if left untreated.",
                'treatment': [
                  "Topical antibiotics: Mupirocin or fusidic acid ointments applied directly to the affected area.",
                  "Oral antibiotics: Prescribed in more severe or widespread cases to eliminate the infection.",
                  "Cleaning the affected area: Gently wash with warm water and antibacterial soap to remove crusts and reduce bacterial load.",
                  "Avoid scratching: Trim nails and cover affected areas to prevent the spread of infection.",
                  "Hygiene practices: Wash affected clothing, towels, and bedding in hot water to kill bacteria.",
                  "Monitor for complications: Seek medical attention if the infection spreads or worsens."
                ],
                'precautions': [
                  "Maintain good hygiene: Regular handwashing and keeping wounds clean can help prevent impetigo.",
                  "Avoid close contact: Keep infected individuals at home until the sores are no longer contagious.",
                  "Disinfect personal items: Avoid sharing personal items like towels, razors, or clothing.",
                  "Cover affected areas: Bandage or cover sores to minimize spreading.",
                  "Prompt wound care: Clean and cover cuts, insect bites, or abrasions to prevent infection.",
                  "Boost immunity: Ensure proper nutrition and rest to strengthen the immune system."
                ],
                'severity': "Mild to moderate. Impetigo usually resolves without complications, but untreated cases can lead to deeper skin infections or more serious conditions. Early treatment reduces the risk of complications and limits the spread of infection."
            },
            'Fungal infection': {
                'description': "Fungal infections are caused by fungi invading and multiplying on the skin, nails, or mucous membranes. Common types include athlete's foot, ringworm, jock itch, and yeast infections. These infections often result in red, itchy, scaly, or cracked skin, and may lead to blisters or discolored, thickened nails. Fungal infections are typically spread through direct contact with infected individuals, animals, or contaminated surfaces. Warm, moist environments encourage fungal growth, increasing the risk of infection in areas like the feet, groin, and under the breasts.",
                'treatment': [
                  "Topical antifungals: Clotrimazole, miconazole, or terbinafine creams applied directly to the affected area.",
                  "Oral antifungals: Prescribed for severe or persistent infections, including fluconazole or itraconazole.",
                  "Antifungal powders and sprays: Help keep affected areas dry and prevent recurrence.",
                  "Good hygiene practices: Wash and dry affected areas thoroughly, especially after sweating.",
                  "Change of clothing: Wear clean, breathable clothing and change socks and underwear frequently.",
                  "Avoid scratching: Minimize irritation to prevent spreading the infection to other areas."
                ],
                'precautions': [
                  "Keep skin dry and clean: Moist environments encourage fungal growth, so thoroughly dry skin after bathing or sweating.",
                  "Wear loose, breathable clothing: Tight or non-breathable fabrics can trap moisture and heat.",
                  "Avoid sharing personal items: Do not share towels, shoes, or clothing with others to reduce the risk of transmission.",
                  "Use antifungal powders: Regularly apply antifungal powders to areas prone to sweating.",
                  "Proper foot care: Wear shower shoes in public showers and locker rooms.",
                  "Boost immune health: Maintain a healthy diet and lifestyle to support the immune system."
                ],
                'severity': "Mild to moderate. While fungal infections are usually not serious, they can cause significant discomfort and may recur if not properly treated. Severe or untreated cases can lead to secondary bacterial infections or widespread skin involvement."
            },
            'Common Cold': {
                'description': "The common cold is a viral infection that affects the upper respiratory tract, including the nose and throat. It is caused by various viruses, most commonly rhinoviruses. Symptoms typically include a runny or stuffy nose, sore throat, coughing, sneezing, mild headache, body aches, and fatigue. The common cold spreads through airborne droplets, direct contact with infected individuals, or touching contaminated surfaces. While generally harmless, it can lead to complications such as sinus infections, ear infections, or exacerbation of asthma.",
                'treatment': [
                  "Rest: Adequate rest helps the body fight off the virus more efficiently.",
                  "Hydration: Drink plenty of fluids to stay hydrated and thin mucus.",
                  "Over-the-counter medications: Decongestants, antihistamines, and pain relievers can alleviate symptoms.",
                  "Saltwater gargle: Helps soothe a sore throat and reduce irritation.",
                  "Humidifiers and steam: Moist air can ease congestion and throat discomfort.",
                  "Vitamin C and zinc: May reduce the duration and severity of cold symptoms if taken early."
                ],
                'precautions': [
                  "Frequent handwashing: Reduces the risk of contracting or spreading the virus.",
                  "Avoid close contact: Stay away from infected individuals to prevent transmission.",
                  "Disinfect surfaces: Regularly clean surfaces that are frequently touched.",
                  "Use tissues and dispose of them properly: Prevents the spread of germs.",
                  "Boost immune system: Maintain a healthy diet, regular exercise, and adequate sleep.",
                  "Avoid touching face: Reduces the chance of the virus entering through the nose, mouth, or eyes."
                ],
                'severity': "Mild. The common cold typically resolves within 7-10 days without complications. However, in some cases, secondary infections such as sinusitis or bronchitis may occur, requiring medical attention."
            },
            'Pneumonia': {
                'description': "Pneumonia is an infection that inflames the air sacs in one or both lungs, causing them to fill with fluid or pus. It can be caused by bacteria, viruses, or fungi. Symptoms may include chest pain, coughing (sometimes with phlegm), fever, chills, and difficulty breathing. Pneumonia can range from mild to severe and is especially dangerous for infants, older adults, and individuals with weakened immune systems. Severe cases may lead to complications such as respiratory failure, sepsis, or lung abscesses.",
                'treatment': [
                  "Antibiotics: Prescribed for bacterial pneumonia to eliminate the infection.",
                  "Antiviral medications: Used for viral pneumonia, though most cases resolve on their own.",
                  "Antifungal medications: For pneumonia caused by fungal infections.",
                  "Fever and pain management: Over-the-counter medications like acetaminophen or ibuprofen.",
                  "Oxygen therapy: Administered in severe cases to maintain adequate oxygen levels.",
                  "Hospitalization: Required for severe pneumonia, particularly in high-risk individuals."
                ],
                'precautions': [
                  "Vaccination: Get pneumococcal and flu vaccines to reduce the risk of pneumonia.",
                  "Good hygiene: Regular handwashing and disinfecting frequently-touched surfaces.",
                  "Avoid smoking: Smoking damages the lungs, increasing the risk of infection.",
                  "Manage chronic conditions: Control underlying conditions like asthma or diabetes.",
                  "Healthy lifestyle: Maintain a strong immune system through proper diet, exercise, and sleep.",
                  "Avoid exposure: Limit contact with sick individuals and wear masks in high-risk environments."
                ],
                'severity': "Moderate to severe. Pneumonia can be life-threatening, particularly in vulnerable populations. Early diagnosis and prompt treatment are crucial to prevent serious complications."
            },
            'Dimorphic Hemorrhoids': {
                'description': "Dimorphic hemorrhoids refer to the simultaneous presence of both internal and external hemorrhoids. Internal hemorrhoids develop inside the rectum, while external hemorrhoids form under the skin around the anus. This condition can cause discomfort, pain, itching, swelling, and bleeding during bowel movements. Severe cases may lead to thrombosis (clotting), prolapse, or infection. Dimorphic hemorrhoids can result from chronic constipation, prolonged sitting, obesity, or straining during bowel movements.",
                'treatment': [
                  "Topical treatments: Over-the-counter creams, ointments, and suppositories to reduce inflammation and pain.",
                  "Warm sitz baths: Sitting in warm water for 15-20 minutes several times a day to alleviate discomfort.",
                  "Dietary changes: Increase fiber intake and hydration to prevent constipation and ease bowel movements.",
                  "Oral pain relievers: Non-prescription pain medications like ibuprofen or acetaminophen.",
                  "Minimally invasive procedures: Rubber band ligation, sclerotherapy, or coagulation for persistent hemorrhoids.",
                  "Surgical intervention: Hemorrhoidectomy or stapling in severe cases that do not respond to other treatments."
                ],
                'precautions': [
                  "Avoid straining: Refrain from excessive straining during bowel movements.",
                  "Maintain hydration: Drink plenty of water to soften stools.",
                  "High-fiber diet: Incorporate fruits, vegetables, and whole grains to promote regular bowel movements.",
                  "Exercise regularly: Physical activity helps prevent constipation and reduces pressure on veins.",
                  "Proper hygiene: Keep the anal area clean and dry to avoid irritation and infection.",
                  "Avoid prolonged sitting: Take breaks and move around to reduce pressure on the rectal area."
                ],
                'severity': "Mild to severe. While dimorphic hemorrhoids can often be managed with home treatments, severe cases may require medical intervention to prevent complications such as thrombosis or persistent bleeding."
            },
            'Arthritis': {
                'description': "Arthritis is a broad term that refers to inflammation of the joints, leading to pain, stiffness, swelling, and decreased range of motion. The most common types include osteoarthritis, which results from wear and tear of cartilage, and rheumatoid arthritis, an autoimmune condition that attacks joint tissues. Arthritis can affect any joint but commonly impacts the knees, hips, hands, and spine. The condition can worsen over time, potentially leading to joint deformity and disability if untreated.",
                'treatment': [
                  "Medications: Nonsteroidal anti-inflammatory drugs (NSAIDs), corticosteroids, and disease-modifying antirheumatic drugs (DMARDs) to reduce inflammation and slow progression.",
                  "Physical therapy: Exercises and stretches to improve joint flexibility, strengthen muscles, and reduce stiffness.",
                  "Weight management: Maintaining a healthy weight reduces stress on weight-bearing joints.",
                  "Assistive devices: Use of canes, braces, or orthotics to support affected joints and improve mobility.",
                  "Hot and cold therapy: Application of heat to relax muscles and cold packs to reduce swelling.",
                  "Surgical options: Joint replacement or repair procedures in severe cases where other treatments fail."
                ],
                'precautions': [
                  "Regular exercise: Engage in low-impact activities like swimming, cycling, and walking to maintain joint function.",
                  "Joint protection: Use proper posture and ergonomic tools to minimize joint strain.",
                  "Healthy diet: Anti-inflammatory foods such as fruits, vegetables, and omega-3 fatty acids can help manage symptoms.",
                  "Avoid overexertion: Balance activity with rest to prevent flare-ups and excessive joint stress.",
                  "Early diagnosis: Seek medical attention at the first sign of joint pain or stiffness to start early intervention.",
                  "Monitor joint health: Regular check-ups and imaging can track disease progression and adjust treatments as needed."
                ],
                'severity': "Mild to severe. Arthritis can range from occasional discomfort to debilitating pain and joint damage. Early treatment and lifestyle modifications can significantly improve quality of life and reduce the risk of long-term complications."

            },
            'Acne': {
                'description': "Acne is a common skin condition that occurs when hair follicles become clogged with oil, dead skin cells, and bacteria. It often results in the formation of pimples, blackheads, whiteheads, and cysts. Acne typically appears on the face, neck, chest, back, and shoulders. While most common during adolescence due to hormonal changes, acne can affect individuals of all ages. Severe acne may lead to scarring, skin discoloration, and emotional distress if left untreated.",
                'treatment': [
                      "Topical treatments: Benzoyl peroxide, salicylic acid, and retinoids to reduce inflammation, unclog pores, and prevent new breakouts.",
                      "Oral medications: Antibiotics, hormonal treatments (birth control), and isotretinoin for more severe or persistent cases.",
                      "Cleansing: Gentle, regular cleansing with non-comedogenic products to keep the skin clean and oil-free.",
                      "Chemical peels and microdermabrasion: Procedures to exfoliate the skin and reduce scarring.",
                      "Laser and light therapies: Help to kill acne-causing bacteria and reduce oil production.",
                      "Steroid injections: Used for large, painful cysts to reduce inflammation quickly."
                ],
                'precautions': [
                      "Avoid picking or squeezing: Picking at acne can worsen inflammation and lead to scarring.",
                      "Regular skin care: Use gentle, oil-free cleansers and moisturizers to prevent clogging pores.",
                      "Balanced diet: Reduce consumption of dairy and high-glycemic foods that may trigger breakouts.",
                      "Stay hydrated: Drinking plenty of water helps maintain skin health.",
                      "Avoid heavy makeup: Use non-comedogenic, water-based cosmetics to reduce pore blockage.",
                      "Manage stress: Stress can exacerbate acne by increasing oil production. Practice stress-reducing techniques."
                ],
                'severity': "Mild to severe. While mild acne often resolves with over-the-counter treatments, severe forms can lead to scarring and significant skin damage if untreated. Professional medical intervention can help manage symptoms and prevent complications."

            },
            'Bronchial Asthma': {
                'description': "Bronchial asthma is a chronic respiratory condition that affects the airways, causing them to become inflamed, narrow, and produce excess mucus. This leads to difficulty in breathing, wheezing, chest tightness, and coughing, particularly during physical activity, at night, or in response to allergens. Asthma can be triggered by allergens, respiratory infections, cold air, smoke, and stress. While asthma cannot be cured, it can be effectively managed with proper treatment and lifestyle adjustments to prevent severe attacks and complications.",
                'treatment': [
                      "Inhaled corticosteroids: Long-term control to reduce inflammation and prevent symptoms.",
                      "Bronchodilators: Short-acting and long-acting medications to relax muscles around the airways.",
                      "Leukotriene modifiers: Oral medications to block chemicals that cause airway tightening.",
                      "Allergy medications: Antihistamines and allergy shots to reduce allergic reactions that can trigger asthma.",
                      "Oxygen therapy: Administered in severe cases to maintain adequate oxygen levels.",
                      "Emergency treatment: Quick-relief inhalers and oral corticosteroids for asthma attacks."
                ],
                'precautions': [
                      "Avoid triggers: Identify and minimize exposure to allergens, smoke, and pollutants.",
                      "Maintain a clean environment: Regular cleaning and air purification to reduce dust mites and pet dander.",
                      "Regular exercise: Engage in controlled, low-impact activities like swimming to improve lung function.",
                      "Follow medication plan: Adhere to prescribed medications even if symptoms are not present.",
                      "Monitor symptoms: Use a peak flow meter to track lung function and detect worsening symptoms early.",
                      "Annual vaccinations: Get flu and pneumonia vaccines to prevent respiratory infections that can exacerbate asthma."
                ],
                'severity': "Mild to severe. Asthma can range from minor, intermittent symptoms to life-threatening attacks. Proper management, early intervention, and regular medical follow-ups are crucial to preventing complications and maintaining quality of life."

            },
            'Hypertension': {
                'description': "Hypertension, also known as high blood pressure, is a chronic medical condition in which the force of blood against the artery walls is consistently too high. Over time, hypertension can lead to severe health issues, including heart disease, stroke, kidney failure, and vision problems. It is often called the 'silent killer' because it may not present noticeable symptoms until significant damage has occurred. Risk factors include obesity, smoking, high salt intake, stress, and lack of physical activity.",
                'treatment': [
                      "Lifestyle changes: Adopt a balanced diet (such as the DASH diet), reduce salt intake, exercise regularly, and maintain a healthy weight.",
                      "Medications: Prescribed antihypertensive drugs including diuretics, ACE inhibitors, beta-blockers, calcium channel blockers, and angiotensin II receptor blockers (ARBs).",
                      "Stress management: Engage in relaxation techniques such as yoga, meditation, and deep breathing exercises.",
                      "Limit alcohol and caffeine: Reduce consumption to lower blood pressure.",
                      "Regular monitoring: Frequently check blood pressure at home and during doctor visits.",
                      "Quit smoking: Ceasing tobacco use lowers the risk of cardiovascular complications."
                ],
                'precautions': [
                      "Healthy diet: Focus on fruits, vegetables, whole grains, and low-fat dairy products.",
                      "Physical activity: Engage in at least 30 minutes of moderate exercise most days of the week.",
                      "Weight management: Maintain a healthy weight to reduce strain on the cardiovascular system.",
                      "Reduce sodium intake: Aim to consume less than 2,300 mg of sodium daily.",
                      "Avoid stress: Implement stress-relieving practices to reduce blood pressure spikes.",
                      "Regular health check-ups: Monitor cardiovascular health regularly and adhere to prescribed treatment plans."
                ],
                'severity': "Mild to severe. Hypertension can silently damage vital organs over time, leading to life-threatening conditions. Early detection, lifestyle adjustments, and medical intervention are essential to prevent long-term complications."

            },
            'Migraine': {
                'description': "Migraine is a neurological condition characterized by intense, throbbing headaches often accompanied by nausea, vomiting, and sensitivity to light and sound. Migraines can last from a few hours to several days and may be preceded by sensory warning symptoms known as aura, which can include visual disturbances, tingling sensations, or difficulty speaking. The exact cause is not fully understood but is believed to involve genetic and environmental factors, with triggers including stress, hormonal changes, lack of sleep, certain foods, and sensory stimuli.",
                'treatment': [
                      "Pain relievers: Over-the-counter medications like ibuprofen, aspirin, and acetaminophen can help alleviate mild migraine symptoms.",
                      "Triptans: Prescription drugs that block pain pathways in the brain and help reduce inflammation.",
                      "Anti-nausea medication: Helps manage nausea and vomiting associated with migraines.",
                      "Preventive medications: Beta-blockers, antidepressants, anticonvulsants, and CGRP inhibitors are prescribed to reduce the frequency and severity of migraines.",
                      "Lifestyle adjustments: Maintain regular sleep, eating, and exercise routines to reduce triggers.",
                      "Alternative therapies: Acupuncture, biofeedback, and relaxation techniques may help manage symptoms."
                ],
                'precautions': [
                      "Avoid known triggers: Identify and steer clear of foods, stressors, or environmental factors that can induce migraines.",
                      "Stay hydrated: Drink plenty of water to prevent dehydration, a common migraine trigger.",
                      "Maintain regular routines: Consistency in sleep patterns and meal times can help prevent migraines.",
                      "Limit screen time: Reduce exposure to bright lights and prolonged screen use.",
                      "Manage stress: Practice stress-relieving techniques such as meditation, yoga, and breathing exercises.",
                      "Track symptoms: Keep a migraine diary to identify patterns and triggers, aiding in better management."
                ],
                'severity': "Mild to severe. While migraines are not life-threatening, they can significantly impair daily activities and quality of life. Chronic migraines can lead to long-term neurological and psychological effects, necessitating early and effective management."
            },
            'Cervical spondylosis': {
                'description': "Cervical spondylosis, also known as neck arthritis, is a condition that affects the bones and discs in the cervical spine (neck). It is commonly caused by age-related wear and tear, leading to the degeneration of the intervertebral discs and the formation of bone spurs. As the discs lose their cushioning ability, it can result in pressure on the spinal cord and nerves, leading to pain, stiffness, and reduced range of motion. While it is most common in older adults, it can also affect younger individuals due to injury or lifestyle factors such as poor posture or repetitive stress.",
                'treatment': [
                    "Medications: Pain relievers such as acetaminophen, NSAIDs, and muscle relaxants to reduce pain and inflammation.",
                    "Physical therapy: Exercises aimed at strengthening the neck muscles, improving posture, and enhancing flexibility.",
                    "Hot and cold therapy: Heat pads to relax tense muscles and cold compresses to reduce inflammation and pain.",
                    "Cervical collar: A soft neck collar to provide support and limit movement, helping reduce strain and alleviate pain.",
                    "Epidural steroid injections: In cases of severe pain, corticosteroid injections can reduce inflammation around the affected nerve roots.",
                    "Surgery: In rare cases, if conservative treatments fail and there is significant nerve compression or damage, surgical intervention such as a discectomy or spinal fusion may be required."
                ],
                'precautions': [
                    "Posture correction: Maintain proper posture to prevent strain on the neck, especially while sitting or working at a desk.",
                    "Ergonomic adjustments: Use ergonomic furniture, such as chairs with proper neck support, to minimize neck strain.",
                    "Regular breaks: Avoid staying in the same position for long periods. Take frequent breaks to stretch and move.",
                    "Weight management: Maintaining a healthy weight can reduce strain on the spine and neck.",
                    "Avoid heavy lifting: Be cautious with lifting heavy objects, especially when twisting or jerking movements can strain the neck.",
                    "Stress management: Practice relaxation techniques like meditation or yoga to reduce muscle tension and stress, which can contribute to neck pain."
                ],
                'severity': "Mild to moderate in most cases. Cervical spondylosis can range from mild discomfort to severe pain, with potential for numbness, tingling, and weakness if nerve compression occurs. Early intervention and lifestyle changes can significantly reduce symptoms and prevent long-term disability. However, in rare cases, if left untreated or undiagnosed, it may lead to chronic pain or nerve damage that requires more invasive treatments."
            },
            'Jaundice': {
                'description': "Jaundice is a medical condition characterized by the yellowing of the skin and the whites of the eyes due to an excess of bilirubin in the bloodstream. Bilirubin is a yellow pigment produced during the breakdown of red blood cells. Jaundice is not a disease in itself but a symptom of underlying conditions, such as liver disease, bile duct obstruction, or hemolytic anemia. It can occur when the liver is unable to process and excrete bilirubin efficiently, or when there is a blockage in the bile ducts preventing bilirubin from being expelled.",
                'treatment': [
                    "Medications: Depending on the underlying cause, medications such as antivirals for viral hepatitis or steroids for autoimmune hepatitis may be prescribed.",
                    "Phototherapy: In cases of newborn jaundice, light therapy helps break down bilirubin in the skin.",
                    "Surgical procedures: If jaundice is caused by a bile duct obstruction, surgery or endoscopic procedures may be required to remove the blockage.",
                    "Blood transfusion: In cases of hemolytic anemia, a blood transfusion may be necessary to replace destroyed red blood cells.",
                    "Liver transplant: In severe cases of liver failure or cirrhosis, a liver transplant may be required if other treatments do not work.",
                    "Supportive care: Adequate hydration, a balanced diet, and rest are important for individuals recovering from liver-related causes of jaundice."
                ],
                'precautions': [
                    "Regular liver function tests: Monitor liver health regularly to detect early signs of liver disease, particularly in high-risk individuals.",
                    "Avoid alcohol: Limit or avoid alcohol consumption to prevent liver damage, especially in individuals with liver disease or jaundice.",
                    "Proper hygiene: Wash hands thoroughly and avoid contaminated food or water to prevent infections that may lead to jaundice.",
                    "Vaccination: Get vaccinated against hepatitis A and B to reduce the risk of viral liver infections.",
                    "Healthy diet: Eat a balanced diet rich in fruits, vegetables, and whole grains to support liver health.",
                    "Avoid unnecessary medications: Be cautious with the use of over-the-counter medications or herbal supplements that could damage the liver."
                ],
                'severity': "Jaundice itself is not a disease but a symptom of underlying conditions, which can range from mild to severe. The severity depends on the root cause, such as liver infections, hepatitis, or bile duct obstructions. If untreated or left undiagnosed, jaundice can lead to liver failure, permanent liver damage, or other serious complications. Early diagnosis and treatment of the underlying cause are crucial to prevent long-term health issues."

            },
            'Malaria': {
                'description': "Malaria is a life-threatening disease caused by Plasmodium parasites, which are transmitted to humans through the bites of infected female Anopheles mosquitoes. The parasites infect and destroy red blood cells, leading to symptoms such as fever, chills, fatigue, headache, and nausea. Malaria is most commonly found in tropical and subtropical regions, particularly in sub-Saharan Africa, Southeast Asia, and parts of South America. If left untreated, it can lead to severe complications such as organ failure, anemia, or death.",
                'treatment': [
                        "Antimalarial medications: Drugs such as chloroquine, artemisinin-based combination therapies (ACTs), and primaquine are commonly used to treat malaria, depending on the Plasmodium species and the severity of the infection.",
                        "Hospitalization: In severe cases, patients may require hospitalization for intravenous administration of antimalarial drugs and supportive care.",
                        "Blood transfusion: If the patient has developed severe anemia due to the destruction of red blood cells, a blood transfusion may be necessary.",
                        "Prevention of relapse: In cases of Plasmodium vivax or Plasmodium ovale malaria, medications such as primaquine are used to target dormant liver forms of the parasite and prevent relapses.",
                        "Supportive care: Fluid and electrolyte management, along with fever control, can help alleviate symptoms and prevent complications."
                ],
                'precautions': [
                        "Use of insect repellent: Apply insect repellents containing DEET or other effective ingredients to prevent mosquito bites.",
                        "Insecticide-treated nets: Sleep under insecticide-treated mosquito nets, especially in malaria-endemic areas, to reduce the risk of mosquito bites.",
                        "Anti-malarial prophylaxis: Travelers to malaria-endemic areas may need to take preventive antimalarial medications as a prophylactic measure.",
                        "Environmental control: Reduce mosquito breeding sites by eliminating standing water, using insecticides, and keeping living areas well-screened.",
                        "Wear protective clothing: When in malaria-prone areas, wear long-sleeved shirts and pants to minimize exposure to mosquitoes.",
                        "Seek medical attention: If symptoms of malaria appear, seek prompt medical attention for diagnosis and treatment to prevent complications."
                ],
                'severity': "Malaria can range from mild to severe. In mild cases, symptoms are flu-like and can resolve with appropriate treatment. However, in severe cases, it can cause life-threatening complications, such as organ failure, cerebral malaria (brain infection), and death. Children under 5 years old, pregnant women, and individuals with weakened immune systems are particularly vulnerable to severe malaria. Early detection and proper treatment are crucial to prevent complications and reduce mortality."

            },
            'urinary tract infection': {
                'description': "A urinary tract infection (UTI) is an infection that can affect any part of the urinary system, including the kidneys, bladder, ureters, and urethra. The majority of UTIs are caused by bacteria, with Escherichia coli (E. coli) being the most common pathogen. UTIs can lead to symptoms such as frequent urination, burning sensation while urinating, cloudy or bloody urine, lower abdominal pain, and fever. While UTIs are more common in women, they can affect anyone, including men and children. If left untreated, UTIs can lead to kidney infections or sepsis, which can be life-threatening.",
                'treatment': [
                        "Antibiotics: The primary treatment for UTIs involves antibiotics such as trimethoprim-sulfamethoxazole, nitrofurantoin, or ciprofloxacin, depending on the severity of the infection and the bacteria involved.",
                        "Pain relievers: Medications like phenazopyridine can be used to alleviate pain, burning, and discomfort while urinating.",
                        "Increased fluid intake: Drinking plenty of water can help flush out bacteria from the urinary system.",
                        "Bladder analgesics: Non-prescription medications like cranberry extracts may help in symptom management, although their effectiveness is debated.",
                        "Hospitalization: In severe cases or for complicated UTIs, hospitalization and intravenous antibiotics may be necessary, especially if there is a risk of kidney damage or infection spread."
                ],
                'precautions': [
                        "Proper hygiene: Wipe from front to back after using the toilet to reduce the risk of bacteria spreading to the urinary tract.",
                        "Urination habits: Urinate frequently and completely, and avoid holding urine for long periods of time.",
                        "Hydration: Drink plenty of water to maintain adequate hydration and help flush out bacteria.",
                        "Urinate after intercourse: Empty the bladder after sexual activity to prevent bacteria from entering the urinary tract.",
                        "Avoid irritating feminine products: Avoid using harsh soaps, douches, or feminine sprays that can irritate the urinary tract and increase the risk of infection.",
                        "Cranberry products: Some studies suggest that cranberry juice or supplements may help prevent UTIs by preventing bacteria from adhering to the urinary tract lining, though more research is needed."
                ],
                'severity': "UTIs can range from mild to severe. Mild cases often present with discomfort or burning while urinating and can be treated with oral antibiotics. If left untreated or in cases of complicated UTIs, the infection can spread to the kidneys, causing more severe symptoms such as fever, back pain, and nausea. Kidney infections can lead to long-term complications, including kidney damage or sepsis. Pregnant women, elderly individuals, and those with compromised immune systems are at higher risk for severe UTIs. Early treatment is essential to avoid complications."

            },
            'allergy': {
                'description': "An allergy is an immune system reaction to a foreign substance (allergen) that is typically harmless to most people. When an allergic person comes into contact with an allergen, their immune system mistakenly identifies it as a threat and produces antibodies to fight it off. Common allergens include pollen, dust mites, animal dander, certain foods, insect stings, and medications. Allergic reactions can vary from mild symptoms, such as sneezing, itching, and skin rashes, to more severe reactions like difficulty breathing, anaphylaxis, or even death in extreme cases.",
                'treatment': [
                  "Antihistamines: Medications such as loratadine, cetirizine, or diphenhydramine block histamine, which causes many allergy symptoms like sneezing, itching, and runny nose.",
                  "Corticosteroids: Nasal or oral steroids like fluticasone or prednisone reduce inflammation in the body, providing relief from allergic reactions, particularly for asthma or rhinitis.",
                  "Decongestants: Medications such as pseudoephedrine or phenylephrine help reduce nasal congestion and improve breathing.",
                  "Allergy shots (immunotherapy): In cases of severe or long-term allergies, allergy shots gradually desensitize the immune system to specific allergens.",
                  "Epinephrine: For severe allergic reactions (anaphylaxis), an injection of epinephrine can be life-saving, and individuals at risk should carry an epinephrine auto-injector.",
                  "Avoidance of allergens: The most effective treatment is avoiding contact with known allergens, such as staying indoors during pollen season or using dust mite covers."
                ],
                'precautions': [
                  "Avoid allergens: Identify and avoid exposure to known allergens, such as certain foods, pets, pollen, or mold.",
                  "Keep indoor air clean: Use air purifiers and maintain a clean environment to reduce exposure to allergens like dust mites, pet dander, and mold.",
                  "Monitor pollen counts: On high pollen days, limit outdoor activities and keep windows closed to reduce exposure to allergens.",
                  "Medications: Take prescribed allergy medications regularly, even during off-seasons, to prevent flare-ups.",
                  "Wear a medical alert bracelet: Individuals with severe allergies should wear a medical alert bracelet that indicates their specific allergies, in case of an emergency.",
                  "Consult an allergist: Get tested for allergies to identify specific triggers and develop a personalized treatment plan."
                ],
                'severity': "Allergic reactions can range from mild to severe. Mild reactions might include sneezing, itching, or skin rashes, which can often be managed with over-the-counter medications. Severe reactions, such as anaphylaxis, can be life-threatening and require immediate medical attention, including the administration of epinephrine. People with a history of severe allergies should always carry an epinephrine auto-injector. Early intervention and proper management can significantly reduce the severity and frequency of allergic reactions."

            },
            'gastroesophageal reflux disease': {
                'description': "Gastroesophageal reflux disease (GERD) is a chronic digestive condition where stomach acid or bile irritates the food pipe (esophagus). It occurs when the lower esophageal sphincter (LES), a ring of muscle at the end of the esophagus, relaxes abnormally or weakens, allowing stomach contents to flow back into the esophagus. Symptoms of GERD include heartburn, regurgitation, chest pain, difficulty swallowing, and a sour taste in the mouth. If left untreated, GERD can lead to complications such as esophageal ulcers, bleeding, or narrowing of the esophagus.",
                'treatment': [
                    "Antacids: Over-the-counter medications such as Tums or Maalox neutralize stomach acid and provide quick relief from heartburn.",
                    "Proton pump inhibitors (PPIs): Medications like omeprazole, lansoprazole, and esomeprazole reduce stomach acid production, helping to heal the esophagus and prevent reflux.",
                    "H2 blockers: Drugs such as ranitidine and famotidine decrease acid production, providing relief from GERD symptoms.",
                    "Prokinetics: Medications like metoclopramide help improve the function of the LES and promote stomach emptying.",
                    "Surgery: In severe cases of GERD that do not respond to medications, a surgical procedure called fundoplication can be performed to tighten the LES and prevent acid reflux.",
                    "Lifestyle changes: Eating smaller meals, avoiding trigger foods, and elevating the head of the bed can help reduce symptoms."
                ],
                'precautions': [
                    "Avoid trigger foods: Limit or eliminate foods and beverages that can relax the LES, such as chocolate, caffeine, alcohol, spicy foods, and fatty meals.",
                    "Eat smaller meals: Eating smaller, more frequent meals helps reduce pressure on the stomach and prevents acid reflux.",
                    "Don't lie down immediately after eating: Wait at least 2-3 hours before lying down to allow the stomach to empty and reduce the risk of acid reflux.",
                    "Lose weight: Excess weight, especially around the abdomen, can increase pressure on the stomach, contributing to GERD.",
                    "Elevate the head of the bed: Raising the head of the bed by 6-8 inches can prevent acid from flowing back into the esophagus during sleep.",
                    "Quit smoking: Smoking weakens the LES and promotes acid reflux, so quitting smoking can help improve GERD symptoms."
                ],
                'severity': "GERD can range from mild to severe. In mild cases, symptoms such as occasional heartburn can be managed with lifestyle changes and over-the-counter medications. Severe cases may cause chronic heartburn, difficulty swallowing, regurgitation of food or liquid, and esophageal damage, potentially leading to complications like esophageal ulcers, bleeding, and an increased risk of esophageal cancer. With proper treatment, GERD can often be managed effectively, but if left untreated, it can result in long-term damage to the esophagus."

            },
            'drug reaction': {
                'description': "A drug reaction occurs when a person's immune system reacts adversely to a medication, causing a range of symptoms. The reaction can be mild, such as a rash or nausea, or severe, such as anaphylaxis, organ failure, or even death. Drug reactions can be due to allergies, toxicity, or side effects. They can occur after the first dose or after prolonged use. Common medications that cause reactions include antibiotics (e.g., penicillin), anticonvulsants, nonsteroidal anti-inflammatory drugs (NSAIDs), and chemotherapy agents. Drug reactions can vary in severity and may require immediate medical attention depending on the type and extent of the reaction.",
                'treatment': [
                    "Discontinuing the drug: The first and most important step is to stop taking the drug that is causing the reaction, after consulting with a healthcare provider.",
                    "Antihistamines: For mild allergic reactions, antihistamines like diphenhydramine can help relieve symptoms such as rashes or itching.",
                    "Corticosteroids: In more severe cases, corticosteroids such as prednisone may be used to reduce inflammation and control symptoms.",
                    "Epinephrine: For severe reactions like anaphylaxis, an injection of epinephrine is crucial to rapidly reverse symptoms and prevent life-threatening complications.",
                    "Supportive care: In cases of severe drug reactions, hospitalization and intravenous fluids or oxygen therapy may be required to stabilize the patient.",
                    "Desensitization: In certain cases, where a drug is necessary but causes a mild allergic reaction, a desensitization protocol may be used under medical supervision."
                ],
                'precautions': [
                    "Inform healthcare providers: Always inform your doctor or pharmacist about any known drug allergies before starting a new medication.",
                    "Read labels: Always read the medication labels and package inserts to be aware of potential side effects and interactions.",
                    "Start with a low dose: When prescribed a new drug, starting with a lower dose and gradually increasing it can help identify any adverse reactions early.",
                    "Monitor for reactions: Keep track of any new symptoms that occur after starting a new medication, and report them to your doctor immediately.",
                    "Wear medical identification: Individuals with known drug allergies should wear a medical alert bracelet or carry identification indicating their allergies.",
                    "Avoid self-medication: Do not take over-the-counter medications without consulting a healthcare provider, especially if you have a history of drug reactions."
                ],
                'severity': "Drug reactions can range from mild to severe. Mild reactions like a rash or mild gastrointestinal discomfort may resolve by discontinuing the drug and treating symptoms. However, severe reactions, such as anaphylaxis, organ failure, or Stevens-Johnson syndrome, can be life-threatening and require immediate emergency care. Early identification of the reaction and prompt medical intervention are crucial to preventing severe outcomes and ensuring proper management."

            },
            'peptic ulcer disease': {
                'description': "Peptic ulcer disease (PUD) refers to the formation of open sores or ulcers in the lining of the stomach, small intestine, or esophagus. The most common cause of PUD is infection with *Helicobacter pylori* bacteria, but it can also be caused by prolonged use of nonsteroidal anti-inflammatory drugs (NSAIDs) or excessive alcohol consumption. The ulcers occur when the protective lining of the digestive tract is damaged, allowing stomach acid to irritate the underlying tissue. Symptoms of PUD include burning stomach pain, bloating, nausea, loss of appetite, and in severe cases, vomiting blood or passing dark stools.",
                'treatment': [
                    "Antibiotics: If *H. pylori* infection is present, antibiotics such as amoxicillin, clarithromycin, or metronidazole are used to eradicate the bacteria.",
                    "Proton pump inhibitors (PPIs): Drugs like omeprazole, lansoprazole, and esomeprazole reduce stomach acid production, promoting ulcer healing and symptom relief.",
                    "H2 blockers: Medications such as ranitidine or famotidine decrease stomach acid and help alleviate ulcer-related symptoms.",
                    "Antacids and sucralfate: These can provide temporary relief by neutralizing stomach acid or forming a protective barrier over the ulcer.",
                    "Avoiding NSAIDs: If the ulcer is caused by NSAID use, discontinuing the medication and switching to alternatives may be necessary.",
                    "Surgery: In rare, severe cases, surgery may be required to repair a perforated ulcer or address complications like bleeding."
                ],
                'precautions': [
                    "Avoid NSAIDs: Nonsteroidal anti-inflammatory drugs like ibuprofen or aspirin should be avoided, as they can irritate the stomach lining and exacerbate ulcers.",
                    "Moderate alcohol consumption: Excessive alcohol can irritate the stomach and delay the healing of ulcers, so limiting alcohol intake is important.",
                    "Quit smoking: Smoking interferes with ulcer healing and increases the risk of developing PUD, so quitting smoking can improve outcomes.",
                    "Healthy diet: Eating smaller, frequent meals and avoiding spicy, acidic, or fatty foods can help reduce stomach acid irritation.",
                    "Manage stress: Chronic stress can increase stomach acid production, so practicing relaxation techniques such as meditation, yoga, or deep breathing can be beneficial.",
                    "Regular medical check-ups: Individuals with a history of ulcers should have regular check-ups to monitor for recurrence or complications."
                ],
                'severity': "Peptic ulcer disease can range from mild to severe. In mild cases, the symptoms are manageable with medications and lifestyle changes. However, severe ulcers can lead to complications such as gastrointestinal bleeding, perforation, or obstruction. If left untreated, PUD can result in life-threatening conditions. Early diagnosis and treatment are essential to manage the condition and prevent long-term damage to the digestive tract."

            },
            'diabetes': {
                'description': "Diabetes is a chronic condition that occurs when the body is unable to properly regulate blood glucose (sugar) levels. There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune condition where the body's immune system attacks and destroys the insulin-producing cells in the pancreas, leading to little or no insulin production. Type 2 diabetes occurs when the body becomes resistant to insulin or when the pancreas does not produce enough insulin. Symptoms of diabetes include excessive thirst, frequent urination, fatigue, blurred vision, and slow-healing sores. Over time, poorly managed diabetes can lead to serious complications such as heart disease, kidney damage, nerve damage, and vision loss.",
                'treatment': [
                    "Insulin therapy: For Type 1 diabetes, insulin injections or an insulin pump are required to regulate blood sugar levels.",
                    "Oral medications: For Type 2 diabetes, medications such as metformin, sulfonylureas, or DPP-4 inhibitors may be used to help control blood sugar.",
                    "Lifestyle changes: A balanced diet, regular exercise, and weight management play a crucial role in managing blood sugar levels, particularly for Type 2 diabetes.",
                    "Blood sugar monitoring: Regular monitoring of blood glucose levels helps adjust medications, insulin doses, and lifestyle changes to maintain target levels.",
                    "Diabetes education: Understanding how to manage diabetes through lifestyle modifications and medication adherence is essential for long-term control.",
                    "Treatment for complications: If complications like neuropathy, retinopathy, or nephropathy develop, specific treatments and interventions will be required to manage them."
                ],
                'precautions': [
                    "Healthy diet: Consume a diet rich in fiber, whole grains, lean proteins, and healthy fats while limiting sugary foods, refined carbohydrates, and processed snacks.",
                    "Regular exercise: Engage in moderate physical activity, such as walking, swimming, or cycling, for at least 150 minutes per week to help improve insulin sensitivity and maintain a healthy weight.",
                    "Monitor blood sugar: Regular blood glucose testing is important to track levels and prevent both hyperglycemia (high blood sugar) and hypoglycemia (low blood sugar).",
                    "Medication adherence: Take prescribed medications or insulin as directed and adjust doses as necessary to keep blood sugar within target range.",
                    "Foot care: People with diabetes are at higher risk of foot infections and ulcers, so daily foot inspection and proper foot care are essential.",
                    "Stress management: Stress can cause fluctuations in blood sugar levels, so practicing relaxation techniques such as meditation or deep breathing can be beneficial."
                ],
                'severity': "Diabetes can range from mild to severe. In well-controlled cases, individuals can lead a normal life with minimal complications. However, if poorly managed, diabetes can lead to serious health complications, including heart disease, stroke, kidney failure, nerve damage, and blindness. The severity of these complications increases with the duration and severity of the disease. Early diagnosis, regular monitoring, and adherence to treatment plans are crucial to prevent or delay complications and ensure better quality of life."

        }
        }

        # Encouraging messages and jokes
        self.jokes = [
            "Why don't skeletons fight each other? They don't have the guts! But you have it to fight with your disease.",
            "You're stronger than you think! Keep fighting, and this too shall pass.",
            "Stay happy and healthy. Let me tell you a joke, Laughter is the best medicine—unless you're dealing with an actual doctor!"
        ]

        self.prepare_data()

    def prepare_data(self):
        """Prepare disease embeddings using ClinicalBERT"""
        disease_descriptions = []
        disease_labels = []

        # Combine symptoms for each disease
        for disease in self.data['label'].unique():
            symptoms = self.data[self.data['label'] == disease]['text'].tolist()
            combined_symptoms = ' '.join(symptoms)
            disease_descriptions.append(combined_symptoms)
            disease_labels.append(disease)

        # Generate embeddings
        self.disease_embeddings = self.get_bert_embeddings(disease_descriptions)
        self.disease_labels = disease_labels

    def get_bert_embeddings(self, texts):
        """Generate BERT embeddings for input texts"""
        embeddings = []

        with torch.no_grad():
            for text in texts:
                inputs = self.tokenizer(text,
                                      return_tensors="pt",
                                      max_length=512,
                                      truncation=True,
                                      padding=True)

                outputs = self.model(**inputs)
                embedding = outputs.last_hidden_state[:, 0, :].numpy()
                embeddings.append(embedding[0])

        return np.array(embeddings)

    def predict_disease(self, user_input):
        """Predict disease based on user symptoms"""
        # Get embedding for user input
        user_embedding = self.get_bert_embeddings([user_input])

        # Calculate similarities
        similarities = cosine_similarity(user_embedding, self.disease_embeddings)

        # Get top matches
        top_indices = similarities[0].argsort()[-3:][::-1]
        predictions = [
            (self.disease_labels[idx], similarities[0][idx])
            for idx in top_indices
        ]

        return predictions

    def get_response(self, user_input):
        """Generate chatbot response based on user input"""
        user_input = user_input.lower().strip()

        # Handle exit commands
        if "quit" in user_input or "bye" in user_input:
            return f"Goodbye {self.user_name if self.user_name else 'user'}! Take care and stay healthy."

        # Handle name introduction
        name_patterns = [
        "i am ([a-zA-Z]+)$",  # Matches "i am John"
        "my name is ([a-zA-Z]+)$",  # Matches "my name is John"
        "^([a-zA-Z]+) here$"  # Matches "John here"
    ]
        for pattern in name_patterns:
          match = re.search(pattern, user_input)
          if match:
            self.user_name = match.group(1)
            return f"Hello {self.user_name}! How are you feeling today?"

        # Handle context-specific queries
        if self.context:
            if "what is it" in user_input or "tell me about it" in user_input:
                return self.disease_info[self.context]['description']
            if "treatment" in user_input:
                return "\n".join(self.disease_info[self.context]['treatment'])
            if "precaution" in user_input:
                return "\n".join(self.disease_info[self.context]['precautions'])
            if "severity" in user_input or "severe" in user_input:
                return self.disease_info[self.context]['severity']
            if "will i die" in user_input or "i am feeling low" in user_input or "i am feeling scared" in user_input:
                return f"Don't worry, {self.user_name}! {np.random.choice(self.jokes)}"

        # Get disease predictions
        predictions = self.predict_disease(user_input)
        predicted_disease, confidence = predictions[0]

        # Set context and generate response
        if confidence < 0.3:
            return "I'm sorry, I couldn't understand your symptoms clearly. Could you please provide more details about what you're experiencing?"

        self.context = predicted_disease
        response = f"Based on your symptoms, {self.user_name if self.user_name else 'user'}, it looks like you might have {predicted_disease}.\n"

        if predicted_disease in self.disease_info:
            response += "Here are some important tips:\n"
            response += "\n".join(self.disease_info[predicted_disease]['treatment'])

        response += "\n\nPlease note: This is not a definitive diagnosis. Always consult a qualified healthcare professional for proper medical evaluation."
        return response


class ChatInterface:
    def __init__(self, chatbot):
        self.chatbot = chatbot
        self.history = []

    def respond(self, message, history):
        try:
            bot_message = self.chatbot.get_response(message)

            # Check if it's a goodbye message
            if "Goodbye" in bot_message:
                return bot_message + "\n\nRefresh the page to start a new chat session."

            return bot_message

        except Exception as e:
            return f"I apologize, but I encountered an error. Please try rephrasing your question.\nError details: {str(e)}"

def initialize_chatbot(data_path):
    """Initialize and return the chatbot instance"""
    print("Loading medical knowledge base and initializing the chatbot...")
    data = pd.read_csv(data_path)
    return EnhancedMedicalChatbot(data)

def launch_interface():
    # Initialize the chatbot
    input_dataset = '/content/drive/MyDrive/715 project/Disease Text Data/Data.csv'  # Update path
    chatbot = initialize_chatbot(input_dataset)

    # Create chat interface
    chat_interface = ChatInterface(chatbot)

    # Configure and launch Gradio interface
    demo = gr.ChatInterface(
        chat_interface.respond,
        chatbot=gr.Chatbot(
            height=400,
            show_label=False,
            show_copy_button=True,
            layout="panel"
        ),
        textbox=gr.Textbox(
            placeholder="Type your symptoms or questions here...",
            show_label=False,
            container=False,
            scale=7
        ),
        title="MediBot",
        description="""Welcome to your personal medical assistant! I'm here to help you understand your symptoms and provide basic medical guidance.""",
        theme="soft",
        examples=[
            "Introduce yourself",
            "Describe symptoms",
            "Treatment?",
            "Severity?",
            "Precautions?"
        ],
        cache_examples=False
    )

    # Launch the interface
    demo.launch(share=True, debug=True)

if __name__ == "__main__":
    launch_interface()

Loading medical knowledge base and initializing the chatbot...


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py:229: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b300a0783c49476763.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b300a0783c49476763.gradio.live
